In [1]:
import os
import shutil
import supervision as sv

root_path = "D:/Downloads/street-facilities-selected"

# 데이터셋 로드
ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{root_path}/rotated-images",
    annotations_directory_path=f"{root_path}/labels-txt",
    data_yaml_path=f"{root_path}/street-facilities.yaml"
)

# 데이터셋 분리 (train 70%, val 15 % test 15%)
train_ds, val_test_ds = ds.split(split_ratio=0.7, random_state=1014, shuffle=True)
val_ds, test_ds = val_test_ds.split(split_ratio=0.5, random_state=535, shuffle=True)

print("train_ds length : ", len(train_ds))
print("val_ds length : ", len(val_ds))
print("test_ds length : ", len(test_ds))

train_ds length :  22827
val_ds length :  4891
test_ds length :  4892


Negative examples 갯수 분석

In [10]:
root_path = "D:/Downloads/street-facilities-selected"
json_path = os.path.join(root_path, "labels")

labels = [f for f in os.listdir(json_path) if f.endswith(".json")]
total_cnt = len(labels)

negative_sample_cnt = [0, 0, 0]
type_cnt_mapping = {
    9 : 0,
    12 : 1,
    13 : 2,
}

for label in labels:
    type = int(label.split("_")[1]) # 09 or 12 or 13
    state = int(label.split("_")[2]) # 0 정상 1 불량
    if state == 0:
        negative_sample_cnt[type_cnt_mapping[type]] += 1

print("total : ", total_cnt)
print("negative examples : ", negative_sample_cnt)

total :  8477
negative examples :  [182, 1083, 915]


### 분할된 데이터셋 클래스 분포 확인

In [2]:
import numpy as np

def count_classes(dataset : sv.DetectionDataset):
    n_classes = len(dataset.classes)
    
    count = np.zeros(n_classes, dtype=np.int16)
    for _, _, annotation in dataset.__iter__():
        # 사진 하나에 존재하는 0개 이상의 class object 들을 각각 센다
        # print(annotation) 
        for label in annotation.class_id:
            count[label] += 1

    return count

# names:
#   0: braille-block-defect
#   1: sidewalk-block-defect
#   2: bicycle-road-defect

print("class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)")
for ds in [train_ds, val_ds, test_ds]:
    print(count_classes(ds))

class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)
[ 7701 10720 11292]
[1487 2332 2437]
[1604 2292 2462]


### 혹시 모를 버그 확인  
당연하지만 빈 라벨 파일에 대해서는 세지 않는다.

In [6]:
# 정상 보도블럭에 대한 라벨 파일. (바운딩 박스 x) 단지 빈 txt 파일이다.
# root_path = "D:/Downloads/street-facilities-selected"
# empty_train_label = f"{root_path}/labels-txt/2_09_0_1_4_1_20210927_0000569294.txt" 

# list = [0, 0, 0]

# with open(empty_train_label, 'r', encoding='utf-8') as f:
#     for newline in f.readlines():
#         list[newline] += 1

# print(list)

저장

In [2]:
# 저장할 디렉토리 경로 설정
root_path = "D:/Downloads/street-facilities-3"
train_images_dir = f"{root_path}/images/train"
train_labels_dir = f"{root_path}/labels/train"

val_images_dir = f"{root_path}/images/val"
val_labels_dir = f"{root_path}/labels/val"

test_images_dir = f"{root_path}/images/test"
test_labels_dir = f"{root_path}/labels/test"

# 디렉토리 생성 (존재하지 않는 경우에만)
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)

os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# DetectionDataset 객체의 이미지 및 라벨을 복사하여 저장
def save_dataset(dataset, images_dir_to_save, labels_dir_to_save):
    i = 1
    total = len(dataset)
    for image_path, image, annotation in dataset.__iter__():
        if i % 200 == 1 : print(f"{i} / {total}")
        # image_path : path/image_filename.jpg or .jpeg
        image_filename = os.path.basename(image_path) # 이미지 파일 이름
        label_filename = os.path.splitext(image_filename)[0] + ".txt" # 확장자명만 변경 (.jpg or .jpeg -> .txt)

        # label_path 조합
        label_path = "\\".join(image_path.split("\\")[:-2] + ["labels-txt", label_filename])
        
        # 이미지 및 라벨을 각각 지정된 폴더로 복사
        shutil.copy(image_path, os.path.join(images_dir_to_save, image_filename))
        shutil.copy(label_path, os.path.join(labels_dir_to_save, label_filename))
        i += 1


# train 데이터셋 저장
save_dataset(train_ds, train_images_dir, train_labels_dir)
# validation 데이터셋 저장
save_dataset(val_ds, val_images_dir, val_labels_dir)
# test 데이터셋 저장
save_dataset(test_ds, test_images_dir, test_labels_dir)

print("Train/Validation/Test 데이터셋 저장 완료.")

1 / 22827
201 / 22827
401 / 22827
601 / 22827
801 / 22827
1001 / 22827
1201 / 22827
1401 / 22827
1601 / 22827
1801 / 22827
2001 / 22827
2201 / 22827
2401 / 22827
2601 / 22827
2801 / 22827
3001 / 22827
3201 / 22827
3401 / 22827
3601 / 22827
3801 / 22827
4001 / 22827
4201 / 22827
4401 / 22827
4601 / 22827
4801 / 22827
5001 / 22827
5201 / 22827
5401 / 22827
5601 / 22827
5801 / 22827
6001 / 22827
6201 / 22827
6401 / 22827
6601 / 22827
6801 / 22827
7001 / 22827
7201 / 22827
7401 / 22827
7601 / 22827
7801 / 22827
8001 / 22827
8201 / 22827
8401 / 22827
8601 / 22827
8801 / 22827
9001 / 22827
9201 / 22827
9401 / 22827
9601 / 22827
9801 / 22827
10001 / 22827
10201 / 22827
10401 / 22827
10601 / 22827
10801 / 22827
11001 / 22827
11201 / 22827
11401 / 22827
11601 / 22827
11801 / 22827
12001 / 22827
12201 / 22827
12401 / 22827
12601 / 22827
12801 / 22827
13001 / 22827
13201 / 22827
13401 / 22827
13601 / 22827
13801 / 22827
14001 / 22827
14201 / 22827
14401 / 22827
14601 / 22827
14801 / 22827
15001 /

저장 완료 후 데이터셋 분포 빠르게 확인

In [4]:
import os 

root_path = "D:/Downloads/street-facilities-3/labels"

sub_path = ["train", "val", "test"]

cnt_classes = [[0, 0, 0] for i in range(3)]

for k, sub in enumerate(sub_path):
    label_path = os.path.join(root_path, sub)
    print(label_path)

    labels = [label for label in os.listdir(label_path) if label.endswith('.txt')]
    total_cnt = len(labels)

    for i, label in enumerate(labels):
        if i % 300 == 0 :
            print(f"{i+1} / {total_cnt}")
        with open(os.path.join(label_path, label), "r+", encoding="utf-8") as f:
            # 기존 내용 읽기 및 수정
            for line in f:
                elems = line.split(" ")
                # 클래스 분포 집계
                cnt_classes[k][int(float(elems[0]))] += 1

print()
for i, type in enumerate(sub_path):
    print(type)
    print(cnt_classes[i])

D:/Downloads/street-facilities-3/labels\train
1 / 26899
301 / 26899
601 / 26899
901 / 26899
1201 / 26899
1501 / 26899
1801 / 26899
2101 / 26899
2401 / 26899
2701 / 26899
3001 / 26899
3301 / 26899
3601 / 26899
3901 / 26899
4201 / 26899
4501 / 26899
4801 / 26899
5101 / 26899
5401 / 26899
5701 / 26899
6001 / 26899
6301 / 26899
6601 / 26899
6901 / 26899
7201 / 26899
7501 / 26899
7801 / 26899
8101 / 26899
8401 / 26899
8701 / 26899
9001 / 26899
9301 / 26899
9601 / 26899
9901 / 26899
10201 / 26899
10501 / 26899
10801 / 26899
11101 / 26899
11401 / 26899
11701 / 26899
12001 / 26899
12301 / 26899
12601 / 26899
12901 / 26899
13201 / 26899
13501 / 26899
13801 / 26899
14101 / 26899
14401 / 26899
14701 / 26899
15001 / 26899
15301 / 26899
15601 / 26899
15901 / 26899
16201 / 26899
16501 / 26899
16801 / 26899
17101 / 26899
17401 / 26899
17701 / 26899
18001 / 26899
18301 / 26899
18601 / 26899
18901 / 26899
19201 / 26899
19501 / 26899
19801 / 26899
20101 / 26899
20401 / 26899
20701 / 26899
21001 / 26899


In [8]:
import numpy as np

arr = np.array([[1901, 5303, 3798],
                [412, 1180, 787],
                [385, 1189, 812]])

print(arr.sum(axis=0))

[2698 7672 5397]


이전 파일 삭제 (optional)

In [9]:
import os
import time

# 삭제할 기준 날짜 설정 (예: '2023-01-01')
delete_before_date = "2024-11-11"

# 기준 날짜를 타임스탬프로 변환
delete_before_timestamp = time.mktime(time.strptime(delete_before_date, "%Y-%m-%d"))

# 대상 폴더 경로
root_path = "D:/Downloads/street-facilities-3"

sub1 = ["images", "labels"]
sub2 = ['test', 'train', 'val']

for d1 in sub1:
    for d2 in sub2:
        folder_path = os.path.join(root_path, d1, d2)
        # 폴더 내 파일 탐색 및 삭제
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            
            # 파일인지 확인 (폴더가 아닌 경우만 삭제)
            if os.path.isfile(file_path):
                # 파일의 수정 시간 가져오기
                file_mod_time = os.path.getmtime(file_path)
                
                # 지정한 날짜 이전에 수정된 파일 삭제
                if file_mod_time < delete_before_timestamp:
                    os.remove(file_path)
                    # print(f"Deleted: {file_path}")
